In [45]:
import time
import os
import pythainlp
import io

from datetime import timedelta
from tqdm import tqdm, tqdm_notebook

print(pythainlp.__version__)
assert pythainlp.__version__ == '2.0.5'

2.0.5


In [3]:
# Load opensubtitle_v2018 TH dataset
PATHS = {
    'opensubtitle_v2018': {
        'th': '../data/opensubtitle_v2018/OpenSubtitles.en-th.th',
        'en': '../data/opensubtitle_v2018/OpenSubtitles.en-th.en'
    }
}

def load_dataset(path):
    with open(path, 'r') as f: 
        return f.read().split('\n')



In [4]:
def test_loaddataset():
    dataset = load_dataset(PATHS['opensubtitle_v2018']['th'])
    print(dataset[0:3])
    assert type(dataset[0]) == str
    
test_loaddataset()    

['ทาสในกระจกวิเศษ, มาจากพื้นที่ที่ไกลที่สุด', 'ผ่านลมและความมืดฉันเรียกเจ้า', 'พูด!']


## Appraoch 1: Use existing pre-trained BPE for Thai language (BPEmb)

BPEmb is a collection of pre-trained subword embeddings in 275 languages, based on Byte-Pair Encoding (BPE) and trained on Wikipedia. Its intended use is as input for neural models in natural language processing. It is based on [SentencePiece](https://arxiv.org/abs/1808.06226) which is an another implemntation of BPE.

Reference: https://github.com/bheinzerling/bpemb



Example code

```
from bpemb import BPEmb

bpemb_th = BPEmb(lang="th", vs=10000)
bpemb_th.encode("เฌอปราง ณรีกุล")

```


In [78]:

from bpemb import BPEmb

bpemb_th_1000 = BPEmb(lang="th", vs=1000)
bpemb_th_1000.encode("เฌอปราง ณรีกุล")


downloading https://nlp.h-its.org/bpemb/th/th.wiki.bpe.vs1000.model


100%|██████████| 251132/251132 [00:01<00:00, 148088.74B/s]


downloading https://nlp.h-its.org/bpemb/th/th.wiki.bpe.vs1000.d100.w2v.bin.tar.gz


100%|██████████| 375702/375702 [00:02<00:00, 186692.29B/s]


['▁เ', 'ฌ', 'อ', 'ป', 'รา', 'ง', '▁', 'ณ', 'รี', 'ก', 'ุ', 'ล']

In [79]:


bpemb_th_5000 = BPEmb(lang="th", vs=5000)
bpemb_th_5000.encode("เฌอปราง ณรีกุล")


['▁เ', 'ฌ', 'อป', 'ราง', '▁ณ', 'รี', 'กุล']

In [80]:

bpemb_th_10000 = BPEmb(lang="th", vs=10000)
bpemb_th_10000.encode("เฌอปราง ณรีกุล")


['▁เ', 'ฌ', 'อป', 'ราง', '▁ณ', 'รี', 'กุล']

In [81]:
bpemb_th_25000 = BPEmb(lang="th", vs=25000)
bpemb_th_25000.encode("เฌอปราง ณรีกุล")


['▁เ', 'ฌ', 'อป', 'ราง', '▁ณ', 'รี', 'กุล']

In [82]:

bpemb_th_100000 = BPEmb(lang="th", vs=100000)
bpemb_th_100000.encode("เฌอปราง ณรีกุล")


['▁เฌ', 'อป', 'ราง', '▁ณ', 'รี', 'กุล']

In [83]:

bpemb_th_200000 = BPEmb(lang="th", vs=200000)
bpemb_th_200000.encode("เฌอปราง ณรีกุล")


['▁เฌ', 'อป', 'ราง', '▁ณ', 'รี', 'กุล']

In [11]:
# It also provide pre-trained subword embedding

print(bpemb_th_25000.most_similar("นักเรียน")[:5])
print('')
print(bpemb_th_25000.most_similar("เรียน")[:5])

[('▁นักเรียน', 0.7826765179634094), ('โรงเรียน', 0.7246325016021729), ('ในโรงเรียน', 0.7122201919555664), ('ของโรงเรียน', 0.6961303353309631), ('ของนักเรียน', 0.672756552696228)]

[('สอน', 0.8173454403877258), ('ศึกษา', 0.7260997295379639), ('ที่โรงเรียน', 0.7104077339172363), ('เข้าเรียน', 0.69277024269104), ('นักเรียน', 0.6714471578598022)]


In [23]:

# Encode opensubtitles Thai Dataset

dataset_opensubtitle_th = load_dataset(PATHS['opensubtitle_v2018']['th'])

In [50]:
print('Examples of BPE encoding for Thai sentences in `opensubtitle_v2018_th`\n')
for index, text in enumerate(dataset_opensubtitle_th[0:30]):
    
    print('{}\nBPE-vocab-5k  : {}\nBPE-vocab-25k : {}\nBPE-vocab-100k: {}\nBPE-vocab-200k: {}\n'.format(text,
                                       bpemb_th_5000.encode(text),
                                       bpemb_th_25000.encode(text),
                                       bpemb_th_100000.encode(text),
                                       bpemb_th_200000.encode(text)))

Examples of BPE encoding for Thai sentences in `opensubtitle_v2018_th`

ทาสในกระจกวิเศษ, มาจากพื้นที่ที่ไกลที่สุด
BPE-vocab-5k  : ['▁ท', 'าส', 'ใน', 'กระจ', 'ก', 'วิ', 'เศษ', ',', '▁', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-vocab-25k : ['▁ท', 'าส', 'ใน', 'กระจก', 'วิเศษ', ',', '▁มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-vocab-100k: ['▁ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', '▁มาจาก', 'พื้นที่ที่', 'ไกล', 'ที่สุด']
BPE-vocab-200k: ['▁ทาส', 'ในกระจก', 'วิเศษ', ',', '▁มาจาก', 'พื้นที่ที่', 'ไกลที่สุด']

ผ่านลมและความมืดฉันเรียกเจ้า
BPE-vocab-5k  : ['▁ผ่าน', 'ลม', 'และความ', 'ม', 'ืด', 'ฉัน', 'เรียก', 'เจ้า']
BPE-vocab-25k : ['▁ผ่าน', 'ลม', 'และความ', 'มืด', 'ฉัน', 'เรียก', 'เจ้า']
BPE-vocab-100k: ['▁ผ่าน', 'ลม', 'และความ', 'มืด', 'ฉัน', 'เรียก', 'เจ้า']
BPE-vocab-200k: ['▁ผ่าน', 'ลม', 'และความ', 'มืด', 'ฉัน', 'เรียก', 'เจ้า']

พูด!
BPE-vocab-5k  : ['▁พ', 'ูด', '!']
BPE-vocab-25k : ['▁พูด', '!']
BPE-vocab-100k: ['▁พูด', '!']
BPE-vocab-200k: ['▁พูด', '!']

ให้ฉันเห็นพระพักตร์ของ พร

In [57]:
sample_texts = [
    'นกกกกกก   กา',
    'รัชานนท์ ชารินทร์',
    'โรงพยาบาลเปาโล Paolo Instition of Medicine'
]
for index, text in enumerate(sample_texts):
    
    print('{}\nBPE-vocab-5k  : {}\nBPE-vocab-25k : {}\nBPE-vocab-100k: {}\nBPE-vocab-200k: {}\n'.format(text,
                                       bpemb_th_5000.encode(text),
                                       bpemb_th_25000.encode(text),
                                       bpemb_th_100000.encode(text),
                                       bpemb_th_200000.encode(text)))

นกกกกกก   กา
BPE-vocab-5k  : ['▁นก', 'ก', 'ก', 'ก', 'ก', 'ก', '▁กา']
BPE-vocab-25k : ['▁นก', 'กก', 'กก', 'ก', '▁กา']
BPE-vocab-100k: ['▁นก', 'กก', 'กก', 'ก', '▁กา']
BPE-vocab-200k: ['▁นก', 'กก', 'กก', 'ก', '▁กา']

รัชานนท์ ชารินทร์
BPE-vocab-5k  : ['▁', 'รัช', 'านนท์', '▁ช', 'าร', 'ินทร์']
BPE-vocab-25k : ['▁รัช', 'านนท์', '▁ช', 'าร', 'ินทร์']
BPE-vocab-100k: ['▁รัช', 'านนท์', '▁ชาร', 'ินทร์']
BPE-vocab-200k: ['▁รัช', 'านนท์', '▁ชาร', 'ินทร์']

โรงพยาบาลเปาโล Paolo Instition of Medicine
BPE-vocab-5k  : ['▁โรงพยาบาล', 'เป', 'า', 'โล', '▁p', 'a', 'ol', 'o', '▁in', 'st', 'ition', '▁of', '▁m', 'ed', 'ic', 'ine']
BPE-vocab-25k : ['▁โรงพยาบาล', 'เปาโล', '▁pa', 'ol', 'o', '▁in', 'st', 'ition', '▁of', '▁medicine']
BPE-vocab-100k: ['▁โรงพยาบาล', 'เปาโล', '▁pa', 'olo', '▁inst', 'ition', '▁of', '▁medicine']
BPE-vocab-200k: ['▁โรงพยาบาลเปาโล', '▁paolo', '▁inst', 'ition', '▁of', '▁medicine']



## Appraoch 2: Build a new BPE vocab with `subword-nmt`

It is different from SentencePiece BPE from BPEmb.


Implementation of `subword-nmt` differs from [Sennrich et al. (2016)](https://arxiv.org/abs/1508.07909)

Example code:

```
subword-nmt learn-bpe -s 1000 < ./data/opensubtitle_v2018/OpenSubtitles.en-th.th > ./data/opensubtitle_v2018/codes.th.symbol-10000

```

Approach:




In [14]:
from subword_nmt import learn_bpe as learner
from subword_nmt import apply_bpe as subword_tokenizer


__1) No Pre-tokenize text__

If the input texts are not pre-tokenized, it treats full sentence as one word. 



In [6]:
num_symbols = 1000
start = time.time()
learner.learn_bpe(
    infile=open('../data/opensubtitle_v2018/OpenSubtitles.en-th.small.th', 'r'),
    outfile=open('../data/opensubtitle_v2018/codes.th.small.no_pretok.symbol-{}'.format(num_symbols), 'w'),
    num_symbols=num_symbols)
end = time.time() - start

print('time taken {}'.format(end))

time taken 0.09339380264282227


no pair has frequency >= 2. Stopping


__2) Pre-tokenized text__


In [7]:
def pretokenize(fobj, total=None):
    """
    Pre-tokenize text input before learning BPE (BPE assume input is pre-tokenized)
        
    Parameters:
        fobj : file object with `rw` mode
        
    Returns: file object like
    """
    
    lines = []
    for i, line in tqdm_notebook(enumerate(fobj), total=total):
        tokens = pythainlp.tokenize.word_tokenize(line, engine='newmm', keep_whitespace=False)
        lines.append(' '.join(tokens))
    
    print('done tokenize, begin create io.StringIO object.')
    out_fobj = io.StringIO('\n'.join(lines))
    return out_fobj

def test_pretokenize_1():
    lines = ['ฉันทำงานบ้าน', 'ฉันทำงานบ้าน Dental']
    fobj = pretokenize(lines)
    
#     print(fobj.read())
    print('read each line')
    for i, line in enumerate(fobj):
        print('line {}: {}'.format(i+1, line))
        for word in line.strip('\r\n ').split(' '):

            print(word)



def test_pretokenize_2():
    
    opensubtitle_dataset = load_dataset(PATHS['opensubtitle_v2018']['th'])[:100]
#     lines = ['ฉันทำงานบ้าน']
    fobj = pretokenize(opensubtitle_dataset)
    print(fobj)
    print(fobj.read())
  


In [8]:
test_pretokenize_1()



done tokenize, begin create io.StringIO object.
read each line
line 1: ฉัน ทำงานบ้าน

ฉัน
ทำงานบ้าน
line 2: ฉัน ทำงานบ้าน Dental
ฉัน
ทำงานบ้าน
Dental


In [9]:

%time test_pretokenize_2()



done tokenize, begin create io.StringIO object.
ทาส ใน กระจก วิเศษ , มาจาก พื้นที่ ที่ ไกล ที่สุด
ผ่าน ลม และ ความมืด ฉัน เรียก เจ้า
พูด !
ให้ ฉัน เห็น พระพักตร์ ของ พระองค์
สิ่ง ที่ เจ้า จะ รู้ ว่า สมเด็จ พระราชินี ของ ฉัน ได้ อย่างไร
กระจก วิเศษ บน ผนัง ผู้ ที่ เป็น สังขาร หนึ่ง ทั้งหมด หรือไม่
ที่ มีชื่อเสียง เป็น ความงาม ของ เจ้า พระ บาท สมเด็จ พระเจ้าอยู่หัว
แต่ ถือเป็น แม่บ้าน ที่ น่ารัก ที่ ฉัน เห็น
ยาจก ไม่ สามารถ ซ่อน พระคุณ อ่อนโยน ของ เธอ
อนิจจา เธอ มี ความเป็นธรรม มากขึ้น กว่า เจ้า
อนิจจา สำหรับ เธอ !
เปิดเผย ชื่อ ของ เธอ
ริมฝีปาก สีแดง กุหลาบ
ผม สี ดำ เป็น ไม้ มะเกลือ
ผิวขาว เหมือน หิมะ
หิมะ สี ขาว !
ต้อง การทราบ ความลับ ?
สัญญา ว่า จะ ไม่ บอก ?
เรา กำลัง ยืน อยู่ ด้วย กันที่ ต้องการ
ทำให้ ความปรารถนา เข้ากัน
นั่น คือ ทั้งหมด ที่ คุณ ต้อง ทำ
และ ถ้า คุณ ได้ยิน มัน สะท้อน
ความปรารถนา ของ คุณ เร็ว ๆ นี้ จะ เป็นจริง ขึ้น มา
ฉัน ต้องการ ฉัน ต้องการ
สำหรับ คน ที่ ฉัน รัก
เพื่อ หา ฉัน ที่จะ หา ฉัน
วันนี้ วันนี้
ฉัน หวัง ว่า ฉัน หวัง
และ ฉัน ฝัน ของ
ใน สิ่ง ที่ ดี ใน สิ่ง ที่ ดี

__Do Pre-tokenize `OpenSubtitles.en-th.small.th` and create BPE with 1000 vocab__


In [10]:
num_symbols = 1000

start = time.time()

# 1. Pre-tokenize `OpenSubtitles.en-th.small.th` first

print('Step 1: Pre-tokenize `OpenSubtitles.en-th.small.th` first')
in_fobj = open('../data/opensubtitle_v2018/OpenSubtitles.en-th.small.th', 'r')

in_fobj_pretokenized = pretokenize(in_fobj)

# print(in_fobj_pretokenized.read())
print('Step 2: Learn BPE on pre-tokenized text. Tokens are separated by space ` `')
learner.learn_bpe(
    infile=in_fobj_pretokenized,
    outfile=open('../data/opensubtitle_v2018/codes.small.th.symbol-{}'.format(num_symbols), 'w'),
    num_symbols=num_symbols)

end = time.time() - start


print('done!')
print('time taken {}'.format(end))


Step 1: Pre-tokenize `OpenSubtitles.en-th.small.th` first



done tokenize, begin create io.StringIO object.
Step 2: Learn BPE on pre-tokenized text. Tokens are separated by space ` `
done!
time taken 0.11494302749633789


no pair has frequency >= 2. Stopping


In [11]:

with open('../data/opensubtitle_v2018/codes.small.th.symbol-{}'.format(num_symbols), 'r') as f:
    for index, vocab in enumerate(f):
        print('{}: {}'.format(index+1, vocab), end='')
        if index +1 == 10:
            break

1: #version: 0.2
2: ั น</w>
3: ฉ ัน</w>
4: ท ี
5: ที ่</w>
6: อ ง</w>
7: เ ป
8: า ม
9: ว ั
10: ร ะ


In [47]:
# timedelta



__Do Pre-tokenize `OpenSubtitles.en-th.th` and create BPE with 1k, 10k, 100k, 250k vocab__


In [65]:
# Do Pre-tokenize `OpenSubtitles.en-th.th` on 10000 vocab
for num_symbols in [5000]:
    start = time.time()

    # 1. Pre-tokenize `OpenSubtitles.en-th.th` first
    print('Step 1: Pre-tokenize `OpenSubtitles.en-th.th` first')
    in_fobj = open('../data/opensubtitle_v2018/OpenSubtitles.en-th.th', 'r')
    in_fobj_pretokenized = pretokenize(in_fobj, total=len(load_dataset(PATHS['opensubtitle_v2018']['th'])))

    elapsed = time.time() - start
    print('time taken for pre-tokenize {}'.format(str(timedelta(seconds=elapsed))))

    start = time.time()

    # print(in_fobj_pretokenized.read())
    print('Step 2: Learn BPE on pre-tokenized text. Tokens are separated by space ` ` (BPE vocab size={})'.format(num_symbols))
    learner.learn_bpe(
        infile=in_fobj_pretokenized,
        outfile=open('../data/opensubtitle_v2018/codes.th.symbol-{}'.format(num_symbols), 'w'),
        num_symbols=num_symbols)

    print('done! BPE-learn (BPE vocab size={})'.format(num_symbols))
    elapsed = time.time() - start
    print('time taken for BPE learn: {}'.format(str(timedelta(seconds=elapsed))))
    


Step 1: Pre-tokenize `OpenSubtitles.en-th.th` first



done tokenize, begin create io.StringIO object.
time taken for pre-tokenize 0:06:14.832238
Step 2: Learn BPE on pre-tokenized text. Tokens are separated by space ` ` (BPE vocab size=5000)
done! BPE-learn (BPE vocab size=5000)
time taken for BPE learn: 0:00:37.955527


In [66]:
for num_symbols in [1000, 5000, 10000, 25000, 100000, 250000]:
    with open('../data/opensubtitle_v2018/codes.th.symbol-{}'.format(num_symbols), 'r') as f:
        for index, vocab in enumerate(f):
            print('{}: {}'.format(index+1, vocab), end='')
            if index +1 == 30:
                break
    print('-'*25)
    print('\n')

1: #version: 0.2
2: ั น</w>
3: อ ง</w>
4: ี ่</w>
5: ไ ม
6: ่ า</w>
7: ฉ ัน</w>
8: ค ุ
9: จ ะ</w>
10: เ ร
11: คุ ณ</w>
12: ท ี่</w>
13: แ ล
14: เ ธ
15: อ ย
16: ไ ด
17: เ ป
18: ว ่า</w>
19: ั บ</w>
20: เ ข
21: ไม ่
22: ั ้
23: า ย</w>
24: ได ้</w>
25: ้ า</w>
26: ื ่
27: ไ ป</w>
28: ่ า
29: เป ็
30: ผ ม</w>
-------------------------


1: #version: 0.2
2: ั น</w>
3: อ ง</w>
4: ี ่</w>
5: ไ ม
6: ่ า</w>
7: ฉ ัน</w>
8: ค ุ
9: จ ะ</w>
10: เ ร
11: คุ ณ</w>
12: ท ี่</w>
13: แ ล
14: เ ธ
15: อ ย
16: ไ ด
17: เ ป
18: ว ่า</w>
19: ั บ</w>
20: เ ข
21: ไม ่
22: ั ้
23: า ย</w>
24: ได ้</w>
25: ้ า</w>
26: ื ่
27: ไ ป</w>
28: ่ า
29: เป ็
30: ผ ม</w>
-------------------------


1: #version: 0.2
2: ั น</w>
3: อ ง</w>
4: ี ่</w>
5: ไ ม
6: ่ า</w>
7: ฉ ัน</w>
8: ค ุ
9: จ ะ</w>
10: เ ร
11: คุ ณ</w>
12: ท ี่</w>
13: แ ล
14: เ ธ
15: อ ย
16: ไ ด
17: เ ป
18: ว ่า</w>
19: ั บ</w>
20: เ ข
21: ไม ่
22: ั ้
23: า ย</w>
24: ได ้</w>
25: ้ า</w>
26: ื ่
27: ไ ป</w>
28: ่ า
29: เป ็
30: ผ ม</w>
--------------------

1: #version: 0.2
2: ั น</w>
3: อ ง</w>
4: ี ่</w>
5: ไ ม
6: ่ า</w>
7: ฉ ัน</w>
8: ค ุ
9: จ ะ</w>
10: เ ร


__Apply BPE on text `OpenSubtitles.en-th.th` with BPE model (1000 vocab)__



In [75]:
BPE_subwordnmt_1000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-1000', 'r+'),
                        separator="_")

BPE_subwordnmt_5000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-5000', 'r+'),
                        separator="_")

BPE_subwordnmt_10000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-10000', 'r+'),
                        separator="_")

BPE_subwordnmt_25000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-25000', 'r+'),
                        separator="_")

BPE_subwordnmt_100000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-100000', 'r+'),
                        separator="_")

BPE_subwordnmt_250000 = subword_tokenizer.BPE(
                        open('../data/opensubtitle_v2018/codes.th.symbol-250000', 'r+'),
                        separator="_")



In [76]:
print('Examples of BPE encoding (subword-nmt) for Thai sentences in `opensubtitle_v2018_th`\n(required pre-tokenized input text)\n')

for index, text in enumerate(dataset_opensubtitle_th[0:30]):
    tokens = pythainlp.tokenize.word_tokenize(text, keep_whitespace=False)
    text_pretokenized = ' '.join(tokens)
    print('\n[{}] text: {}'.format(index+1, text))
    print('    text_pretokenized:', tokens)
    bpe_tokens = BPE_subwordnmt_1000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-1k : {}'.format(bpe_tokens), end='')
    
    bpe_tokens = BPE_subwordnmt_5000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-5k : {}'.format(bpe_tokens), end='')
    
    bpe_tokens = BPE_subwordnmt_10000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-10k : {}'.format(bpe_tokens), end='')
    
    bpe_tokens = BPE_subwordnmt_25000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-25k : {}'.format(bpe_tokens), end='')
    
    
    bpe_tokens = BPE_subwordnmt_100000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-100k : {}'.format(bpe_tokens), end='')
    
    bpe_tokens = BPE_subwordnmt_250000.segment_tokens(tokens)
#     bpe_tokens = segmented_text.split('_ ')
    print('\nBPE-(subword-nmt)-vocab-250k : {}'.format(bpe_tokens), end='\n')

Examples of BPE encoding (subword-nmt) for Thai sentences in `opensubtitle_v2018_th`
(required pre-tokenized input text)


[1] text: ทาสในกระจกวิเศษ, มาจากพื้นที่ที่ไกลที่สุด
    text_pretokenized: ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']

BPE-(subword-nmt)-vocab-1k : ['ท_', 'า_', 'ส', 'ใน', 'กระ_', 'จ_', 'ก', 'วิ_', 'เศษ', ',', 'มาจาก', 'พ_', 'ื_', '้น_', 'ที่', 'ที่', 'ไ_', 'ก_', 'ล', 'ที่สุด']
BPE-(subword-nmt)-vocab-5k : ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-(subword-nmt)-vocab-10k : ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-(subword-nmt)-vocab-25k : ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-(subword-nmt)-vocab-100k : ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']
BPE-(subword-nmt)-vocab-250k : ['ทาส', 'ใน', 'กระจก', 'วิเศษ', ',', 'มาจาก', 'พื้นที่', 'ที่', 'ไกล', 'ที่สุด']

[2]

## Comparison between two types


In [119]:
print('Comparison: Two BPE implementation (SentencePiece adn subword-nmt learned from opensubtitle). BPE encoding for arbitraty Thai texts\n')

texts = [
    'ฮาจิเมะ มะชิเตะ เอโดงาวะ โคนัน',
    'สุนัขพันธุ์ชิสุ shih tzu อยู๋ในบ้านชั้นสอง',
    'แคทเธอรีน',
    'อนาคอนดา ไพธอน',
    'ปฐมภูมิ ทุติยภูมิ ตติยภูมิ ',
    'เครื่องราชอิสริยาภรณ์',
    'ถ้าพูดถึง การเรียนแพทย์ สิ่งที่เราจะไม่กล่าวถึงไม่ได้เลยคือ ห้องกรอส--ห้องกรอส(Gross) ',
    'วันพรุ่งนี้ฉันจะไปพบประธานาธิบดี โดนัลด์ ทรัมป์ พร้อมกับท่าน',
    'วันพรุ่งนี้ฉันจะไปพบประธานาธิบดี โดนัลด์ ทรัมป์ พร้อมกับท่าน',
    'เริ่มรู้สึกเย็นจนยะเยือกขึ้น ',
    'โจโฉแตกทัพเรือ เป็นภาพยนตร์จีนอิงประวัติศาสตร์',
    'ตาแหวนเป็นหาดหนึ่งในเกาะล้าน ฉันชอบไปเที่ยวหาดตาแหวนและหาดเทียนเป็นประจำ'
]
for index, text in enumerate(texts):
    
    
    print('[{}]: {}'.format(index+1, text))
    print('--- BPE SentencePiece --')
    print('BPE-vocab-5k  : {}\nBPE-vocab-25k : {}\nBPE-vocab-100k: {}\nBPE-vocab-200k: {}\n'.format(
                                       bpemb_th_5000.encode(text),
                                       bpemb_th_25000.encode(text),
                                       bpemb_th_100000.encode(text),
                                       bpemb_th_200000.encode(text)))
    
    tokens = pythainlp.tokenize.word_tokenize(text, engine='newmm', keep_whitespace=False)
    print('--- BPE subword-nmt --')
    print('pre-tokenized text (newmm): {}'.format(' '.join(tokens)))
    print('BPE-(subword-nmt)-vocab-1k : {}'.format( BPE_subwordnmt_1000.segment_tokens(tokens)), end='')
    print('\nBPE-(subword-nmt)-vocab-5k : {}'.format(BPE_subwordnmt_5000.segment_tokens(tokens)), end='')
    print('\nBPE-(subword-nmt)-vocab-10k : {}'.format( BPE_subwordnmt_10000.segment_tokens(tokens)), end='')
    print('\nBPE-(subword-nmt)-vocab-25k : {}'.format(BPE_subwordnmt_25000.segment_tokens(tokens)), end='')
    print('\nBPE-(subword-nmt)-vocab-100k : {}'.format(BPE_subwordnmt_100000.segment_tokens(tokens)), end='')
    print('\nBPE-(subword-nmt)-vocab-250k : {}'.format(BPE_subwordnmt_250000.segment_tokens(tokens)), end='\n')
    print('\n')

Comparison: Two BPE implementation (SentencePiece adn subword-nmt learned from opensubtitle). BPE encoding for arbitraty Thai texts

[1]: ฮาจิเมะ มะชิเตะ เอโดงาวะ โคนัน
--- BPE SentencePiece --
BPE-vocab-5k  : ['▁ฮ', 'า', 'จิ', 'เมะ', '▁มะ', 'ชิ', 'เต', 'ะ', '▁เอ', 'โด', 'ง', 'าว', 'ะ', '▁โ', 'คน', 'ัน']
BPE-vocab-25k : ['▁ฮา', 'จิ', 'เมะ', '▁มะ', 'ชิ', 'เตะ', '▁เอ', 'โด', 'ง', 'าวะ', '▁โคน', 'ัน']
BPE-vocab-100k: ['▁ฮาจิเมะ', '▁มะ', 'ชิ', 'เตะ', '▁เอ', 'โดง', 'าวะ', '▁โคนัน']
BPE-vocab-200k: ['▁ฮาจิเมะ', '▁มะ', 'ชิ', 'เตะ', '▁เอโดงาวะ', '▁โคนัน']

--- BPE subword-nmt --
pre-tokenized text (newmm): ฮา จิ เมะ มะ ชิ เตะ เอ โด งา วะ โค นัน
BPE-(subword-nmt)-vocab-1k : ['ฮ_', 'า', 'จ_', 'ิ', 'เม_', 'ะ', 'ม_', 'ะ', 'ช_', 'ิ', 'เต_', 'ะ', 'เอ', 'โ_', 'ด', 'ง_', 'า', 'วะ', 'โ_', 'ค', 'น_', 'ัน']
BPE-(subword-nmt)-vocab-5k : ['ฮา', 'จิ', 'เม_', 'ะ', 'มะ', 'ชิ', 'เตะ', 'เอ', 'โด', 'งา', 'วะ', 'โค', 'นัน']
BPE-(subword-nmt)-vocab-10k : ['ฮา', 'จิ', 'เม_', 'ะ', 'มะ', 'ชิ', 'เตะ', 'เอ', 'โด', 'งา'